In [ ]:
# 주택 가격예측

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import joblib

# 1. 데이터 로드
file_path = '국토교통부_주택 공시가격 정보(2024).csv'
df = pd.read_csv(file_path)

# 필요한 열 선택
features = ['시도', '시군구', '전용면적']
target = '공시가격'

# 결측치 제거
df.dropna(subset=features + [target], inplace=True)

# 2. 원-핫 인코딩 (ColumnTransformer 대신)
X = df[features]
y = df[target]

# pandas의 get_dummies 활용 → 범주형 변수 자동 원-핫 인코딩
X = pd.get_dummies(X, columns=['시도', '시군구'], drop_first=True)

# 3. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
#  추가 할 때 
# 3-1. 데이터 스케일링 추가
from sklearn.preprocessing import StandardScaler

# 입력 특성(X) 스케일링: '전용면적' 열만 찾아서 스케일링
# 원-핫 인코딩 후 '전용면적'은 보통 첫 번째 열로 남아있음
# 더 안전하게는 열 이름을 직접 지정하거나 Index를 활용해야 합니다.
# 여기서는 '전용면적' 열이 0번째 인덱스에 있다고 가정하고 진행합니다.


# 4-2. 출력 타겟(y) 스케일링: 공시가격 (MAE를 해석하기 쉽게 만들기 위해)
# MSE를 사용하기 때문에 타겟도 스케일링하면 학습이 안정적임
scaler_y = StandardScaler()
# y_train과 y_test는 Series이므로 numpy 배열로 변환 후 2차원 형태로 reshape 필요
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)) # 전치 행렬이 됨
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))


# 4. 텐서플로우 모델 구축
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 모델 요약
model.summary()

# 5. 모델 학습
modelpath="house_price_model.h5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    X_train,  y_train_scaled , # y_train
    epochs=100, # 충분히 큰 값으로 설정 (Early Stopping이 자동으로 종료시키도록)
    batch_size=32,
    validation_split=0.1,
    verbose=1,
    # 콜백 리스트 추가
    callbacks=[early_stopping_callback, checkpointer] 
)


# 학습 완료 후 모델 저장
#model.save("house_price_model.h5")
#print("모델이 저장되었습니다.")


# scaler 저장용
import joblib  
joblib.dump(scaler_y, "scaler_y.pkl")
print("스케일러 저장 완료.")

In [ ]:
#  재 학습
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import matplotlib.pyplot as plt

# 1. 데이터 로드
file_path = '국토교통부_주택 공시가격 정보(2024).csv'
df = pd.read_csv(file_path)

# 필요한 열 선택
features = ['시도', '시군구', '전용면적']
target = '공시가격'

# 결측치 제거
df.dropna(subset=features + [target], inplace=True)

# 2. 원-핫 인코딩 (ColumnTransformer 대신)
X = df[features]
y = df[target]

# pandas의 get_dummies 활용 → 범주형 변수 자동 원-핫 인코딩
X = pd.get_dummies(X, columns=['시도', '시군구'], drop_first=True)

# 3. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
#  추가 할 때 
# 3-1. 데이터 스케일링 추가
from sklearn.preprocessing import StandardScaler

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# 4. 모델 불러오기
model = tf.keras.models.load_model("house_price_model.h5")
print("저장된 모델을 불러왔습니다.")

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae']) 

# 5. 모델 학습
modelpath="house_price_model.h5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    # StandardScaler를 사용하는 경우
    X_train, y_train_scaled , 
    
    # StandardScaler를 사용하지 않는 경우
    # X_train, y_train,
    
    epochs=100, # 충분히 큰 값으로 설정 (Early Stopping이 자동으로 종료시키도록)
    batch_size=32,
    validation_split=0.1,
    verbose=1,
    # 콜백 리스트 추가
    callbacks=[early_stopping_callback, checkpointer] 
)



# 학습 완료 후 모델 저장
#model.save("house_price_model.h5")
#print("모델이 저장되었습니다.")

# 예측 코드

In [ ]:
# scaler_y = StandardScaler()를 사용하지 않았을 때의 예측 코드
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
# 새로운 데이터 준비
new_data = pd.DataFrame([['서울특별시', '종로구', 187.4]], 
                        columns=['시도', '시군구', '전용면적'])

# 학습 데이터에서 사용한 원-핫 인코딩 기준으로 맞추기
new_data_encoded = pd.get_dummies(new_data, columns=['시도', '시군구'], drop_first=True)

# 학습할 때 사용한 X의 컬럼과 동일하게 맞춰주기 (없는 컬럼은 0으로 채우기)
new_data_encoded = new_data_encoded.reindex(columns=X.columns, fill_value=0)

# 모델 로드 (이미 학습된 모델이 있다면)
model = load_model("house_price_model.h5")

# 예측
prediction = model.predict(new_data_encoded)
print("예측된 공시가격:", prediction[0][0])

In [ ]:
# scaler_y = StandardScaler()를 사용 했을 때의 예측 코드
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib

#새로운 데이터 준비
new_data = pd.DataFrame([['서울특별시', '종로구', 187.4]], 
                        columns=['시도', '시군구', '전용면적'])

#학습 시 사용한 인코딩 방식 동일하게 적용
new_data_encoded = pd.get_dummies(new_data, columns=['시도', '시군구'], drop_first=True)

#학습 시 사용된 X 컬럼 구조와 동일하게 맞추기 (없는 컬럼은 0으로 채움)
new_data_encoded = new_data_encoded.reindex(columns=X.columns, fill_value=0)

#저장된 모델 로드
model = load_model("house_price_model.h5")
scaler_y = joblib.load("scaler_y.pkl")

#예측 (스케일된 값이 나옴)
y_pred_scaled = model.predict(new_data_encoded)

#역스케일링 (원래 공시가격 단위로 변환)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

print("예측된 공시가격:", y_pred[0][0])